# Prepare Google Colab Environment

In [ ]:
!git clone https://github.com/mhamilton723/STEGO.git

Cloning into 'STEGO'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 201 (delta 18), reused 15 (delta 12), pack-reused 169
Receiving objects: 100% (201/201), 9.23 MiB | 45.02 MiB/s, done.
Resolving deltas: 100% (98/98), done.


In [1]:
!pip install wget
!pip install torchmetrics
!pip install hydra-core
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git
!pip install omegaconf
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lucasb-eyer/pydensecrf.git to /tmp/pip-req-build-q6ovmarg
  Running command git clone -q https://github.com/lucasb-eyer/pydensecrf.git /tmp/pip-req-build-q6ovmarg
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pydensecrf: filename=pydensecrf-1.0rc2-cp37-cp37m-linux_x86_64.whl size=2782046 sha256=71473ff2b3a4363bda9ceb8f46de25bb51b6d2b7727d03a5ff5f312642aa3e63
  Stored in directory: /tmp/pip-ephem-wheel-cache-uvn8sinm/wheels/c1/7e/80/99adc0b2f215180486e24dd9c700028343ba5f5

# Download Pretrained Model



In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
import os
from os.path import join
import shutil
os.chdir("/content/drive/MyDrive/STEGO/src")
saved_models_dir = join("..", "saved_models")
os.makedirs(saved_models_dir, exist_ok=True)
#shutil.copy("../checkpoints/litter_dataset/directory_exp1_date_Jun28_09-42-44/epoch=2-step=2400.ckpt", os.path.join(saved_models_dir,"trained_model.ckpt"))
#shutil.copy("../checkpoints/litter_dataset/directory_exp2_date_Jul06_08-10-48/epoch=3-step=2400.ckpt", os.path.join(saved_models_dir,"trained_model_exp2.ckpt"))
shutil.copy("../checkpoints/litter_dataset/directory_exp3_date_Jul09_14-45-58/epoch=3-step=2400.ckpt", os.path.join(saved_models_dir,"trained_model_exp3.ckpt"))
saved_model_name = "trained_model_exp3.ckpt"

In [ ]:
#import wget
#saved_model_url_root = "https://marhamilresearch4.blob.core.windows.net/stego-public/saved_models/"
#saved_model_name = "cocostuff27_vit_base_5.ckpt"
#if not os.path.exists(join(saved_models_dir, saved_model_name)):
#  wget.download(saved_model_url_root + saved_model_name, join(saved_models_dir, saved_model_name))

# Load pretrained STEGO

In [9]:
from train_segmentation import LitUnsupervisedSegmenter

model = LitUnsupervisedSegmenter.load_from_checkpoint(join(saved_models_dir, saved_model_name)).cuda()

Loading vit tiny pretrained Dino model
Pretrained weights found at /content/drive/MyDrive/output_dino/checkpoint0090.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (UnsupervisedMetrics). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


### Create composite images with predictions from cluster and linear probe

In [10]:
from utils import unnorm, remove_axes
import torch.nn.functional as F
from crf import dense_crf
import torch
from PIL import Image
import requests
from io import BytesIO
from torchvision.transforms.functional import to_tensor
from utils import get_transform
import glob
from skimage.io import imsave
import pandas as pd 

In [11]:
df_lab = pd.read_csv('/content/drive/My Drive/all_labeled.csv', index_col=0)
X, Y , imfilename, area = [], [], [], []
patch_path = '/content/drive/My Drive/root_patches/patches/'
for i in range(len(df_lab)):
    f = os.path.basename(df_lab.loc[i,'patch_filename']).replace(".JPG", "").split('_')
    df_lab.loc[i,'patch_filename']=os.path.join(patch_path, os.path.basename(df_lab.loc[i,'patch_filename']))
    imfilename.append('_'.join(f[1:3]) + ".JPG")
    area.append(f[0])
    X.append(int(f[3].replace("X", "")))
    Y.append(int(f[4].replace("Y", "")))
df_lab['X'] = X
df_lab['Y'] = Y

df_lab['area'] = area
df_lab['imfilename'] = imfilename
df_lab = df_lab.sort_values(by='imfilename').reset_index(drop=True)
df_lab

,Unnamed: 0.1,index,patch_filename,plastic,X,Y,area,imfilename
0,1874,2412.0,/content/drive/My Drive/root_patches/patches/a...,0.0,4608,512,a1,DJI_0813.JPG
1,1971,2551.0,/content/drive/My Drive/root_patches/patches/a...,0.0,2560,2304,a1,DJI_0813.JPG
2,1948,2527.0,/content/drive/My Drive/root_patches/patches/a...,0.0,1792,2048,a1,DJI_0813.JPG
3,767,2465.0,/content/drive/My Drive/root_patches/patches/a...,1.0,2048,1280,a1,DJI_0813.JPG
4,1831,2363.0,/content/drive/My Drive/root_patches/patches/a...,0.0,2816,0,a1,DJI_0813.JPG
...,...,...,...,...,...,...,...,...
7051,2583,4653.0,/content/drive/My Drive/root_patches/patches/a...,1.0,3072,2816,a2,DJI_0965.JPG
7052,2584,4654.0,/content/drive/My Drive/root_patches/patches/a...,1.0,3328,2816,a2,DJI_0965.JPG
7053,2585,4655.0,/content/drive/My Drive/root_patches/patches/a...,1.0,3584,2816,a2,DJI_0965.JPG
7054,2569,4639.0,/content/drive/My Drive/root_patches/patches/a...,0.0,4864,2560,a2,DJI_0965.JPG


In [15]:
exp = 'exp3_res128'
if not os.path.exists(os.path.join('/content/drive/My Drive/output_stego_trained/',exp)):
  os.makedirs(os.path.join('/content/drive/My Drive/output_stego_trained/', exp, 'cluster_composites'))
  os.makedirs(os.path.join('/content/drive/My Drive/output_stego_trained/', exp, 'probe_composites'))

In [18]:
import numpy as np
from skimage.io import imread 

imgs_path = '/content/drive/My Drive/selected_50m/'
imgs = np.unique(df_lab['imfilename'])
for imf in imgs:
    for a in np.unique(df_lab['area']):
        df = df_lab[(df_lab['imfilename']==imf) & (df_lab['area']==a)].reset_index(drop=True)
        if len(df)!=0:
            im = imread(os.path.join(imgs_path, a, imf))
            comp_clus = np.zeros(im.shape)
            comp_lin_prob = np.zeros(im.shape)
            for i in range(len(df)):
                img = Image.open(df.loc[i, 'patch_filename'])
                x, y = df.loc[i,'X'], df.loc[i,'Y']
                transform = get_transform(256, False, "center")
                img = transform(img).unsqueeze(0).cuda()

                with torch.no_grad():
                  code1 = model(img)
                  code2 = model(img.flip(dims=[3]))
                  code  = (code1 + code2.flip(dims=[3])) / 2
                  code = F.interpolate(code, img.shape[-2:], mode='bilinear', align_corners=False)
                  linear_probs = torch.log_softmax(model.linear_probe(code), dim=1).cpu()
                  cluster_probs = model.cluster_probe(code, 2, log_probs=True).cpu()

                  single_img = img[0].cpu()
                  linear_pred = dense_crf(single_img, linear_probs[0]).argmax(0)
                  cluster_pred = dense_crf(single_img, cluster_probs[0]).argmax(0)
                  cluster_img = model.label_cmap[cluster_pred]
                  linear_prob_img = model.label_cmap[linear_pred]

                  comp_lin_prob[y: y+256 , x: x+256] = linear_prob_img
                  comp_clus[y: y+256 , x: x+256] = cluster_img
            imsave(os.path.join('/content/drive/My Drive/output_stego_trained/', exp, 'cluster_composites/cluster_pred_'+ imf), comp_clus)
            imsave(os.path.join('/content/drive/My Drive/output_stego_trained/', exp, 'probe_composites/linear_prob_'+ imf), comp_lin_prob)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


KeyboardInterrupt: ignored

# Visualize Result

Note that cluster predictions will not be matched with ground truth classes in this example. For hungarian matching please use: `eval_segmentation.py`

In [ ]:
import matplotlib.pyplot as plt
from utils import unnorm, remove_axes
import torch.nn.functional as F
from crf import dense_crf
import torch
from PIL import Image
import requests
from io import BytesIO
from torchvision.transforms.functional import to_tensor
from utils import get_transform
import glob

imgs_paths = glob.glob('/content/drive/My Drive/predicted_patches/litter/*.JPG')
for imfile in imgs_paths:
  if not os.path.exists('/content/drive/My Drive/output_stego_trained/stego_'+ os.path.basename(imfile)):
    img = Image.open(imfile)
    transform = get_transform(256, False, "center")
    img = transform(img).unsqueeze(0).cuda()

    with torch.no_grad():
      code1 = model(img)
      code2 = model(img.flip(dims=[3]))
      code  = (code1 + code2.flip(dims=[3])) / 2
      code = F.interpolate(code, img.shape[-2:], mode='bilinear', align_corners=False)
      linear_probs = torch.log_softmax(model.linear_probe(code), dim=1).cpu()
      cluster_probs = model.cluster_probe(code, 2, log_probs=True).cpu()

      single_img = img[0].cpu()
      linear_pred = dense_crf(single_img, linear_probs[0]).argmax(0)
      cluster_pred = dense_crf(single_img, cluster_probs[0]).argmax(0)

    fig, ax = plt.subplots(1,3, figsize=(5*3,5))
    ax[0].imshow(unnorm(img)[0].permute(1,2,0).cpu())
    ax[0].set_title("Image")
    ax[1].imshow(model.label_cmap[cluster_pred])
    ax[1].set_title("Cluster Predictions")
    ax[2].imshow(model.label_cmap[linear_pred])
    ax[2].set_title("Linear Probe Predictions")
    remove_axes(ax)
    if not os.path.exists('/content/drive/My Drive/output_stego_trained/'):
      os.mkdir('/content/drive/My Drive/output_stego_trained/')
    plt.savefig('/content/drive/My Drive/output_stego_trained/stego_'+ os.path.basename(imfile))

Output hidden; open in https://colab.research.google.com to view.